In [12]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [13]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
        
        

In [ ]:
  
class Log_reg(nn.Module):
    def __init__(self):
        super(Log_reg, self).__init__()
        self.fc1 = nn.Linear(784, 10)
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = F.log_softmax(self.fc1(x))
        return x

In [8]:
class two_layer_net(nn.Module):
    def __init__(self):
        super(two_layer_net, self).__init__()
        self.fc1 = nn.Linear(784, 500)
        self.fc2 = nn.Linear(500, 10)
        self.nonlinearity = F.relu
        
    def forward(self, x):
        x = x.view(-1, 784)
        x = self.nonlinearity(self.fc1(x))
        x = F.log_softmax(self.fc2(x))
        return x

In [3]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 6400 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [4]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [14]:
      
    
use_cuda = torch.cuda.is_available()
#use_cuda = False
lr = 0.001
momentum = 0.5

epochs = 10
torch.manual_seed(2)
model_type="network"

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=64, shuffle=True, **kwargs)

if model_type == "network":
    model = Net().to(device)
elif model_type == "two_layer":
    model = two_layer_net().to(device)
else:
    model = Log_reg().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
        
if (model_type != "network"):
    thing = model.fc1
        
    

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.330464

Test set: Average loss: 0.1642, Accuracy: 9491/10000 (95%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.347047

Test set: Average loss: 0.1082, Accuracy: 9659/10000 (97%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.297553

Test set: Average loss: 0.0803, Accuracy: 9755/10000 (98%)



In [ ]:
import numpy as np
print(dir(model.fc1))
w = model.fc1.weight
mats = []
for i in range(10):
    weights = w[i].reshape((28,28)).cpu().data.numpy()
    weights[np.greater(weights, 0.03)] = 1
    weights[np.less(weights, -0.03)] = -1
    weights[np.less(weights, 0.03) * np.greater(weights, -0.03)] = 0
    mats.append(weights)
    
import matplotlib
import matplotlib.pyplot as plt

for i in range(10):
    plt.figure()
    plt.imshow(mats[i], cmap='Greys')
    plt.colorbar()
    plt.show()